<a href="https://www.kaggle.com/code/sagorkumarmitra/q-a-chatbot-with-python?scriptVersionId=193478659" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pickle
import numpy as np

with open('/kaggle/input/qa-bot/train_qa.txt','rb') as f1:
    train_data = pickle.load(f1)

with open('/kaggle/input/qa-bot/test_qa.txt','rb') as f2:
    test_data = pickle.load(f2)

In [2]:
type(train_data)

list

In [3]:
type(test_data)

list

In [4]:
len(train_data)

10000

In [5]:
len(test_data)

1000

In [6]:
test_data[0]

(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'kitchen', '?'],
 'no')

In [7]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [8]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
all_data = test_data + train_data

In [10]:
len(all_data)

11000

In [11]:
all_data[0]

(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'kitchen', '?'],
 'no')

In [12]:
vocab = set()

for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
    
vocab.add('no')
vocab.add('yes')

In [13]:
len(vocab)

37

In [14]:
vocab_len = len(vocab) + 1

In [15]:
all_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [16]:
# Longest Story

all_story_lens = [len(data[0]) for data in all_data]

max_story_len = max(all_story_lens)

In [17]:
max_question_len = max([len(data[1]) for data in all_data])

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

2024-08-21 18:07:56.015737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 18:07:56.015896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 18:07:56.144249: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
tokenizer.word_index

{'john': 1,
 'sandra': 2,
 'got': 3,
 'grabbed': 4,
 'football': 5,
 'journeyed': 6,
 'up': 7,
 '.': 8,
 'there': 9,
 'hallway': 10,
 '?': 11,
 'put': 12,
 'garden': 13,
 'travelled': 14,
 'picked': 15,
 'the': 16,
 'bedroom': 17,
 'down': 18,
 'apple': 19,
 'yes': 20,
 'office': 21,
 'went': 22,
 'no': 23,
 'is': 24,
 'daniel': 25,
 'discarded': 26,
 'bathroom': 27,
 'took': 28,
 'to': 29,
 'back': 30,
 'mary': 31,
 'in': 32,
 'moved': 33,
 'milk': 34,
 'dropped': 35,
 'kitchen': 36,
 'left': 37}

In [20]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [21]:
# train_data[0]

In [22]:
# train_story_text[0]

In [23]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [24]:
len(train_story_seq)

10000

In [25]:
len(tokenizer.word_index)

37

In [26]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    X = [] # Story
    Xq = [] # Questions
    Y = [] # Answer
    
    for story,query,sanswer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        
        y = np.zeros(len(word_index)+1)
        
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

In [27]:
inputs_train,queries_train,answers_train = vectorize_stories(train_data)

In [28]:
inputs_train

array([[ 0,  0,  0, ..., 16, 17,  8],
       [ 0,  0,  0, ..., 16, 10,  8],
       [ 0,  0,  0, ..., 16, 27,  8],
       ...,
       [ 0,  0,  0, ..., 16, 17,  8],
       [ 0,  0,  0, ..., 34,  9,  8],
       [ 0,  0,  0, ..., 19,  9,  8]], dtype=int32)

In [29]:
queries_train

array([[24,  2, 32, 16, 10, 11],
       [24, 25, 32, 16, 27, 11],
       [24, 25, 32, 16, 21, 11],
       ...,
       [24,  2, 32, 16, 10, 11],
       [24, 31, 32, 16, 36, 11],
       [24, 31, 32, 16, 17, 11]], dtype=int32)

In [30]:
answers_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [32]:
inputs_test

array([[ 0,  0,  0, ..., 16, 17,  8],
       [ 0,  0,  0, ..., 16, 13,  8],
       [ 0,  0,  0, ..., 16, 13,  8],
       ...,
       [ 0,  0,  0, ..., 16, 19,  8],
       [ 0,  0,  0, ..., 16, 13,  8],
       [ 0,  0,  0, ..., 19,  9,  8]], dtype=int32)

In [33]:
tokenizer.word_index['yes']

20

In [34]:
tokenizer.word_index['no']

23

In [35]:
sum(answers_test)

array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0., 1000.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.])